In [7]:
from __future__ import division

import numpy as np
from dipy.core.gradients import gradient_table
import dipy.reconst.shm as shm
import nibabel as nib
from dipy.data.fetcher import read_bvals_bvecs
import scipy.io
import scipy
import numpy
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import matplotlib.pyplot as plt
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from keras import optimizers
from keras.initializers import RandomUniform
from sklearn.decomposition import PCA, KernelPCA
from keras.layers import Dropout

In [8]:
def sh_fit_adc(data, mask, bvals, bvecs, order, regularise=0.006, \
                is_mrtrix=True, flipX=False, flipY=False, flipZ=False, scale=1):
    """
    Spherical Harmonic fit to the normalised signal E.

    Returns:
        shimg: SH fit image # because Camino fits the ADC
    """
    print ('Estimating ADC in SH basis...')
    gtab = gradient_table(bvals, bvecs, b0_threshold=24)
    Esig = shm.normalize_data(data, gtab.b0s_mask)
    Sind = np.where(gtab.b0s_mask==False)[0]
    b = gtab.bvals[Sind]
    g = gtab.gradients[Sind, :]
    if flipX:
        g[:, 0] = -g[:, 0]
    if flipY:
        g[:, 1] = -g[:, 1]
    if flipZ:
        g[:, 2] = -g[:, 2]
    r, th, ph = shm.cart2sphere(g[:,0], g[:,1], g[:,2])
    if is_mrtrix:
        print("SH basis: mrtrix")
        B, m, n = shm.real_sym_sh_mrtrix(order, th, ph)
        print(m)
        print(n)
    else:
        print("SH basis: dipy")
        B, m, n = shm.real_sym_sh_basis(order, th, ph)

    L = n*n*(n+1)*(n+1) * regularise
    Binv = shm.smooth_pinv(B, L)

    shimg = np.zeros((mask.shape[0], mask.shape[1], mask.shape[2], B.shape[1]), \
            dtype=float)
    mindx = np.argwhere(mask>0)
    for i, j, k in mindx:
        E = -np.log(Esig[i, j, k, Sind]) * 1/b
        shimg[i, j, k, :] = np.matmul(Binv, E)

    return shimg*scale

In [9]:
def convert_to_camino(df):
    output = pandas.DataFrame()
    output[0] = 0 # exit
    output[1] = 0
    output[2] = df[0] # c00
    output[3] = df[3]
    output[4] = df[2] 
    output[5] = df[4]
    output[6] = df[1]
    output[7] = df[5]
    output[8] = df[10]
    output[9] = df[9]
    output[10] = df[11]
    output[11] = df[8]
    output[12] = df[12]
    output[13] = df[7]
    output[14] = df[13]
    output[15] = df[6]
    output[16] = df[14]
    output[0] = 0
    output[1] = 0
    return output

In [10]:
# Create mask for training database
mask = np.ones((10929,1,1))
print(mask.shape)

for s in range(1,36):
    
    
    print('shell ' + str(s))
    
    # Load bvals and bvec files
    bvals, bvecs = read_bvals_bvecs('shell_' + str(s) + '.bvals', 'shell_' + str(s) + '.bvecs')
    
    # Load training DB NF signals per shell
    data = scipy.io.loadmat('Signals_NF_shell' + str(s) + '.mat')
    shell = data['shell']
    shell = np.expand_dims(shell, axis = 1)
    shell = np.expand_dims(shell, axis = 1)

    # call the function that fits the spherical harmonics
    shimg = sh_fit_adc(shell, mask, bvals, bvecs, 4, is_mrtrix = True)
    shimg = np.squeeze(shimg)
    
    # convert into camino units
    shimg = shimg * 1e-6

    # save the fit
    data = {}
    data['shimg_R0_006_S1'] = (convert_to_camino(pandas.DataFrame(shimg))).as_matrix()
    scipy.io.savemat('signals_shimg_shell' + str(s) +'_R0_006_S1.mat',data)


(10929, 1, 1)
shell 1
Estimating ADC in SH basis...
SH basis: mrtrix
[ 0  2  1  0 -1 -2  4  3  2  1  0 -1 -2 -3 -4]
[0 2 2 2 2 2 4 4 4 4 4 4 4 4 4]
shell 2
Estimating ADC in SH basis...
SH basis: mrtrix
[ 0  2  1  0 -1 -2  4  3  2  1  0 -1 -2 -3 -4]
[0 2 2 2 2 2 4 4 4 4 4 4 4 4 4]
shell 3
Estimating ADC in SH basis...
SH basis: mrtrix
[ 0  2  1  0 -1 -2  4  3  2  1  0 -1 -2 -3 -4]
[0 2 2 2 2 2 4 4 4 4 4 4 4 4 4]
shell 4
Estimating ADC in SH basis...
SH basis: mrtrix
[ 0  2  1  0 -1 -2  4  3  2  1  0 -1 -2 -3 -4]
[0 2 2 2 2 2 4 4 4 4 4 4 4 4 4]
shell 5
Estimating ADC in SH basis...
SH basis: mrtrix
[ 0  2  1  0 -1 -2  4  3  2  1  0 -1 -2 -3 -4]
[0 2 2 2 2 2 4 4 4 4 4 4 4 4 4]
shell 6
Estimating ADC in SH basis...
SH basis: mrtrix
[ 0  2  1  0 -1 -2  4  3  2  1  0 -1 -2 -3 -4]
[0 2 2 2 2 2 4 4 4 4 4 4 4 4 4]
shell 7
Estimating ADC in SH basis...
SH basis: mrtrix
[ 0  2  1  0 -1 -2  4  3  2  1  0 -1 -2 -3 -4]
[0 2 2 2 2 2 4 4 4 4 4 4 4 4 4]
shell 8
Estimating ADC in SH basis...
SH basis: 

In [11]:
shell = scipy.io.loadmat('signals_shimg_shell1_R0_006_S1.mat')
shell = pandas.DataFrame(shell['shimg_R0_006_S1'])
print(shell)

        0    1             2             3             4             5   \
0      0.0  0.0  4.142238e-09  1.421184e-09  5.673825e-12  4.515276e-12   
1      0.0  0.0  4.048756e-09  1.411367e-09 -9.668507e-12 -5.666505e-12   
2      0.0  0.0  2.670240e-09  9.122979e-10 -1.095429e-12 -2.764851e-12   
3      0.0  0.0  1.932207e-09  6.470675e-10 -3.377105e-12 -8.741951e-12   
4      0.0  0.0  2.746550e-09  9.260697e-10 -5.755862e-12  2.596017e-12   
5      0.0  0.0  4.159418e-09  1.438084e-09 -5.540238e-12  3.585636e-12   
6      0.0  0.0  4.030248e-09  1.383951e-09  8.052746e-12  1.866467e-12   
7      0.0  0.0  2.666229e-09  9.038272e-10  4.539040e-12 -6.757815e-12   
8      0.0  0.0  1.928482e-09  6.398064e-10  2.154235e-12  2.016302e-12   
9      0.0  0.0  2.760335e-09  9.359413e-10  6.656719e-12  5.228014e-12   
10     0.0  0.0  4.158524e-09  1.430727e-09 -6.143888e-12 -9.538391e-12   
11     0.0  0.0  4.062735e-09  1.387872e-09  4.419070e-12  8.221613e-12   
12     0.0  0.0  2.683103

In [12]:
shell

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,0.0,4.142238e-09,1.421184e-09,5.673825e-12,4.515276e-12,6.938929e-12,1.946050e-12,-6.062720e-13,4.652094e-13,2.937246e-13,3.722079e-13,3.548362e-13,-2.420598e-13,3.246890e-14,2.284276e-14,-1.686439e-13
1,0.0,0.0,4.048756e-09,1.411367e-09,-9.668507e-12,-5.666505e-12,-1.082861e-12,1.172746e-11,-5.580057e-13,3.885113e-13,2.803345e-13,4.085143e-13,3.860181e-13,-2.188901e-13,4.814711e-14,-1.317641e-14,-1.841096e-13
2,0.0,0.0,2.670240e-09,9.122979e-10,-1.095429e-12,-2.764851e-12,-1.741714e-12,-4.214556e-12,-2.782395e-13,2.426865e-13,1.554405e-13,2.438237e-13,2.654332e-13,-1.041959e-13,4.557585e-14,8.624810e-15,-7.782355e-14
3,0.0,0.0,1.932207e-09,6.470675e-10,-3.377105e-12,-8.741951e-12,5.002984e-12,-3.357714e-12,-1.648609e-13,1.629022e-13,9.763985e-14,1.897633e-13,1.824093e-13,-6.623094e-14,4.009409e-14,2.544058e-15,-5.160124e-14
4,0.0,0.0,2.746550e-09,9.260697e-10,-5.755862e-12,2.596017e-12,-7.064379e-12,6.771859e-12,-3.156009e-13,2.359932e-13,1.554906e-13,2.601700e-13,2.686593e-13,-1.224272e-13,4.777746e-14,-7.794210e-15,-1.015341e-13
5,0.0,0.0,4.159418e-09,1.438084e-09,-5.540238e-12,3.585636e-12,5.149217e-12,-4.301704e-12,-5.607780e-13,4.205433e-13,2.974927e-13,3.988066e-13,3.399149e-13,-2.164244e-13,5.217748e-14,1.398848e-14,-1.659078e-13
6,0.0,0.0,4.030248e-09,1.383951e-09,8.052746e-12,1.866467e-12,-4.585767e-12,-2.622602e-13,-5.177284e-13,3.910102e-13,2.920265e-13,4.135739e-13,3.945241e-13,-2.018120e-13,5.852389e-14,-3.594713e-15,-1.608613e-13
7,0.0,0.0,2.666229e-09,9.038272e-10,4.539040e-12,-6.757815e-12,3.594335e-13,-2.277532e-12,-3.054685e-13,2.638947e-13,1.334067e-13,2.558389e-13,2.323615e-13,-1.061503e-13,3.613741e-14,6.859683e-15,-8.704080e-14
8,0.0,0.0,1.928482e-09,6.398064e-10,2.154235e-12,2.016302e-12,-1.324181e-12,5.648821e-12,-1.554069e-13,1.752507e-13,9.716123e-14,1.680856e-13,1.612215e-13,-7.803764e-14,2.989551e-14,-6.295543e-15,-5.612835e-14
9,0.0,0.0,2.760335e-09,9.359413e-10,6.656719e-12,5.228014e-12,-4.354014e-12,1.620430e-12,-3.144706e-13,2.794921e-13,1.704475e-13,2.769169e-13,2.608503e-13,-1.183871e-13,3.886702e-14,-2.886710e-15,-9.155129e-14
